[View in Colaboratory](https://colab.research.google.com/github/serereuk/tobigs/blob/master/%ED%88%AC%EB%B9%85%EC%8A%A4_8%EC%A3%BC%EC%B0%A8.ipynb)

In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmpz312atms/pubring.gpg' created
gpg: /tmp/tmpz312atms/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [5]:
!ls -ltr

total 12
drwxr-xr-x 2 root root 4096 Sep  6 17:46 sample_data
-rw-r--r-- 1 root root 2550 Sep 10 11:15 adc.json
drwxr-xr-x 2 root root 4096 Sep 10 11:32 drive


In [6]:
%cd drive

/content/drive


In [7]:
!ls -ltr

total 26
drwxr-xr-x 2 root root 4096 Aug 31 03:03 Colab Notebooks
drwxr-xr-x 2 root root 4096 Sep 10 02:35 제목없는 폴더
drwxr-xr-x 2 root root 4096 Sep 10 02:41 Dataset
-rw-r--r-- 1 root root 7643 Sep 10 10:48 Building Classifier.ipynb
-rw-r--r-- 1 root root 5716 Sep 10 11:31 Untitled0.ipynb


In [8]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

In [10]:
training_set = train_datagen.flow_from_directory('Dataset/train',
target_size = (256,256),
class_mode = 'categorical')

Found 24201 images belonging to 3 classes.


In [11]:
test_set = test_datagen.flow_from_directory('Dataset/test',
target_size = (256,256),                                            
class_mode = 'categorical')

Found 70 images belonging to 3 classes.


In [0]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout

In [0]:
custom = Sequential()

In [0]:
custom.add(Conv2D(filters = 32, 
                  kernel_size = (3,3),
                  input_shape = (256,256,3),
                  strides = (1,1),
                  padding = 'same',
                  activation = 'relu'))

In [0]:
custom.add(MaxPooling2D(pool_size = (2, 2)))

In [0]:
custom.add(Conv2D(filters = 64, 
                  kernel_size = (3,3),
                  input_shape = (256,256,3),
                  strides = (1,1),
                  padding = 'same',
                  activation = 'relu'))

In [0]:
custom.add(MaxPooling2D(pool_size = (2, 2)))

In [0]:
custom.add(Conv2D(filters = 32, 
                  kernel_size = (3,3),
                  input_shape = (256,256,3),
                  strides = (1,1),
                  padding = 'same',
                  activation = 'relu'))

In [0]:
custom.add(Flatten())

In [0]:
custom.add(Dense(units = 500, activation = 'sigmoid'))

In [0]:
custom.add(Dropout(0.5))

In [0]:
custom.add(Dense(units = 3, activation = 'softmax'))

In [0]:
custom.compile(optimizer = 'Adam', loss = 'categorical_crossentropy')

In [40]:
custom.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 64, 32)        18464     
_________________________________________________________________
flatten_2 (Flatten)          (None, 131072)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)               65536500  
__________

In [41]:
custom.fit_generator(training_set,
                     epochs = 10,
                     validation_data = test_set,
                     steps_per_epoch = 10)

Epoch 1/10
10/10 [==============================] - 239s 24s/step - loss: 1.9022 - val_loss: 1.9711
Epoch 2/10
10/10 [==============================] - 162s 16s/step - loss: 0.9629 - val_loss: 1.7143
Epoch 3/10
10/10 [==============================] - 211s 21s/step - loss: 1.1388 - val_loss: 1.2866
Epoch 4/10
10/10 [==============================] - 190s 19s/step - loss: 1.1054 - val_loss: 1.3485
Epoch 5/10
10/10 [==============================] - 189s 19s/step - loss: 0.9725 - val_loss: 1.4739
Epoch 6/10
10/10 [==============================] - 191s 19s/step - loss: 1.0828 - val_loss: 1.3831
Epoch 7/10
10/10 [==============================] - 188s 19s/step - loss: 0.9687 - val_loss: 1.3117
Epoch 8/10
10/10 [==============================] - 196s 20s/step - loss: 0.9548 - val_loss: 1.4132
Epoch 9/10
10/10 [==============================] - 200s 20s/step - loss: 0.9837 - val_loss: 1.4403
Epoch 10/10
10/10 [==============================] - 206s 21s/step - loss: 0.9875 - val_loss: 1.3587

Q1. Input에 3*3 convolution filter 6개를 돌렸을 때(stride=1, padding=0) activation map의 dimension은? 그리고 이때 filter 6개의 depth는 몇 일까요?

A1. activation map = [718, 1278, 6] depth = 3

Q2. Input에 3*3 convolution filter 10개를 돌렸을 때(stride=1, padding=1) activation map의 dimension은? 그리고 이때 학습하는 총 weight는 몇 개일까요? (bias 제외)

A2. 3 X 3 X 3 X 10 = 270

Q3. Q2의 결과인 activation map에 1*1 convolution filter 5개를 돌렸을 때(stride=1, padding=0) activation map의 dimension은? 그리고 이때 filter 5개의 depth는 몇 일까요?

A3. [720, 1280, 5] depth =10

Q4. Q2의 결과인 activation map에 2*2 max pooling filter을 적용할 때 (stride=2) activation map의 dimension은?

A4. [360, 640, 5] 조심스럽게 예상해봅니다



1. ILSVRC / PASCAL VOC / MS COCO / Early Stopping / Transfer Learning / GPGPU



*   ILSVRC = Imagenet Large Scale Visual Recognition Challenge  
외국판 이미지 공모전인듯 하다 2010년부터 진행되어 오고 있다.
*   PASCAL VOC =  Pattern Analysis Statistical Modelling and Computational Learning
외국판 이미지 공모전2, 2005년부터 진행되어 오고 있다.
*   MSCOCO = common objects in contexts
이미지 segementation 공모전 
*   Early Stopping = 오버피팅을 방지하기 위해서 적당히 학습이 된다고 판단되면 정지하는 기법
*   Transfer Learning = 기존의 만들어진 모델을 사용하여 새로운 모델을 만들시 학습을 빠르게 하며, 예측을 더 높이는 방법입니다. 이유는 시간을 줄이고 효율성을 늘리기 위해서
*   GPGPU = gpu 친구들 행렬 계산에 특화되어 있당 게임하고 싶습니다





2. Alexnet의 정확한 Input size(227 or 224)를 구하고 그렇게 생각하는 이유에 대해 적어주세요. 그리고 그 모델을 Sequential Model로 구현해주세요.

사실 곰곰이 생각해보면 227이 맞다. 집에 가면서 stride랑 그런걸 계산해보니 55 차원이 나오려면 227이 나와야한다
하지만  데이터 augmentation의 측면에서 수많은 데이터를 형성하기 위해서,  2048의 숫자를 맞추려면 224도 맞다. pytorch는 floor를 좋아해서 소수점을 버려서 나온 결과가 모델이 돌아갔당. 심심해서 텐서플로우 공부하는 겸 저자가 직접 만든 텐서플로우 코드를 확인해본 결과 227로 인풋을 넣었당. 저자도 문제점을 확인하고 고친 모양인듯하다.

Full (simplified) AlexNet architecture:
[227x227x3] INPUT   
[55x55x96] CONV1: 96 11x11 filters at stride 4, pad 0  
[27x27x96] MAX POOL1: 3x3 filters at stride 2   
[27x27x96] NORM1: Normalization layer  
[27x27x256] CONV2: 256 5x5 filters at stride 1, pad 2   
[13x13x256] MAX POOL2: 3x3 filters at stride 2     
[13x13x256] NORM2: Normalization layer  
[13x13x384] CONV3: 384 3x3 filters at stride 1, pad 1  
[13x13x384] CONV4: 384 3x3 filters at stride 1, pad 1  
[13x13x256] CONV5: 256 3x3 filters at stride 1, pad 1  
[6x6x256] MAX POOL3: 3x3 filters at stride 2  
[4096] FC6: 4096 neurons  
[4096] FC7: 4096 neurons  
[1000] FC8: 1000 neurons (class scores)  

In [0]:
hi_alex = Sequential()

In [0]:
hi_alex.add(Conv2D(filters = 96, 
                  kernel_size = (11,11),
                  input_shape = (227,227,3),
                  strides = (4,4),
                  padding = 'valid',
                  activation = 'relu'))
hi_alex.add(MaxPooling2D(pool_size = (2, 2), strides = (2,2)))
hi_alex.add(keras.layers.ZeroPadding2D(padding=(2, 2))) # 패딩 사이즈 설정 아주 불편하다 그리워요 텐서플로우
hi_alex.add(Conv2D(filters = 256, 
                  kernel_size = (5,5),
                  input_shape = (31,31,96),
                  strides = (1,1),
                  padding = "valid",
                  activation = 'relu'))
hi_alex.add(MaxPooling2D(pool_size = (3, 3), strides = (2,2)))
hi_alex.add(Conv2D(filters = 384, 
                  kernel_size = (3,3),
                  input_shape = (13,13,192),
                  strides = (1,1),
                  padding = "same",
                  activation = 'relu'))
hi_alex.add(Conv2D(filters = 384, 
                  kernel_size = (3,3),
                  input_shape = (13,13,192),
                  strides = (1,1),
                  padding = "same",
                  activation = 'relu'))
hi_alex.add(Conv2D(filters = 256, 
                  kernel_size = (3,3),
                  input_shape = (13,13,128),
                  strides = (1,1),
                  padding = "same",
                  activation = 'relu'))
hi_alex.add(MaxPooling2D(pool_size = (3, 3), strides = (2,2)))


In [0]:
hi_alex.add(Flatten())
hi_alex.add(Dense(units = 4096, activation = 'sigmoid'))
hi_alex.add(Dense(units = 4096, activation = 'sigmoid'))
hi_alex.add(Dense(units = 1000, activation = 'softmax'))

In [69]:
hi_alex.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
zero_padding2d_6 (ZeroPaddin (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 13, 13, 384)       885120    
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 13, 13, 384)       1327488   
__________